According to MIT Climate Portal, the urban heat island effect is a phenomenon whereby cities experience higher air temperatures than the surrounding countryside. Scientists refer to areas afflicted by these higher temperatures as urban heat islands, and people living in these areas are particularly exposed to the effects of climate change. (https://climate.mit.edu/explainers/urban-heat-islands).

Moreover, certain social groups are particularly vulnerable to crises, including climate change, for example, **children**, persons with disabilities, ethnic minorities, **migrant workers**, **displaced persons**, **older people**, and other socially marginalized groups. (https://www.worldbank.org/en/topic/social-dimensions-of-climate-change#1).

If we focus our attention on migrants (and in particular refugees) and displaced people, the UNHCR (UN Refugee Agency) states that the vast majority of refugees (approximately 78 percent) do not live in camps but in cities, and are therefore more vulnerable to the extreme heatwaves caused by climate change. In general urban locations offer more opportunities to live autonomously and find employment, but they also pose major challenges as refugees are often forced to share accommodation or live in non-functional public buildings, collective centers, slums or other types of informal settlements with substandard living conditions. (https://action.unrefugees.org/refugee-facts/camps/)

THe UNHCR provides a database with the demographics of **forcibly displaced population** including internally dispaced people, refugees, asylum-seekers and other people in need of international protection that settled in urban locations in the countries of asylum. Since there is no universal definition of urban, UNCHR classifies an urban location as a settlement with more than 5,000 inhabitants.(2021, 2022 Data: https://www.unhcr.org/refugee-statistics/download/?url=0quh0N)

The data were then filtered to identify the developing countries among the asylum countries according to the UN historical classification of developing and developed countries in 2022 and 2021 through a VLOOKUP:
https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Funstats.un.org%2Funsd%2Fmethodology%2Fm49%2Fhistorical-classification-of-developed-and-developing-regions.xlsx&wdOrigin=BROWSELINK


Data were cleaned to have data for both 2021 and 2022 for all the developing countries.
Major **urban refugees settlements** were then identified through web scraping from wikipedia (https://en.wikipedia.org/wiki/Urban_refugee#References). Data were harmonised for Country name and ISO to have consistent data among the used database.

Coordinates of the cities in Developing countries with urban refugees settlements were found to then identify:
* Highest temperatures 2013 - 2023 to identify temperature variations trend and anomalies,
* Days with temperatures >= 35° Celsius (that, according to WHO, may cause heat-related diseases, especially if combined to high humidity https://www.who.int/news-room/fact-sheets/detail/climate-change-heat-and-health).

According to WHO, infants or people who are over 60 years of age or that have chronic health conditions are those most affected by **heat-related diseases**, therefore the focus will be on developing countries hosting urban forcibly displaced population where babies < 4 years old and elderly > 60 years old are at least => 50 (Data filtered through Pivot Table)

In [4]:
#Check if we have access to the .csv

cat Cities.csv

Country,Country ISO,Urban area
Armenia,ARM,Yerevan
Azerbaijan,AZE,Baku
Burkina Faso,BFA,Ouagadougou
Burundi,BDI,Bujumbura
Congo,COG,Brazzaville
Cote d'Ivoire,CIV,Abidjan
Dem. Rep. of the Congo,COD,Kinshasa
Egypt,EGY,Alexandria
Egypt,EGY,Cairo
Egypt,EGY,Giza
Ethiopia,ETH,Addis Ababa
Ghana,GHA,Accra
India,IND,Delhi
Indonesia,IDN,Jakarta
Iraq,IRQ,Babylon
Iraq,IRQ,Baghdad
Thailand,THA,Bangkok
Turkey,TUR,Ankara
Uganda,UGA,Kampala
Syrian Arab Rep.,SYR,Damascus
Senegal,SEN,Dakar
Libya,LBY,Tripoli
Morocco,MAR,Rabat
Mexico,MEX,Mexico City
Sudan,SDN,Kassala
South Sudan,SSD,Juba
Turkey,TUR,Istanbul
Pakistan,PAK,Islamabad
Somalia,SOM,Hargeisa
Zimbabwe,ZWE,Harare
Afghanistan,AFG,Kabul
Niger,NER,Niamey
Kenya,KEN,Nairobi
Mozambique,MOZ,Maputo
Nigeria,NGA,Lagos
Malaysia,MYS,Kuala Lumpur
Rwanda,RWA,Kigali
Sudan,SDN,Khartoum
Venezuela (Bolivarian Republic of),VEN,Caracas
Yemen,YEM,Aden
Yemen,YEM,'Amran
Yemen,YEM,Sana'a
Yemen,YEM,Mukalla


In [8]:
#Test the API

curl "https://nominatim.openstreetmap.org/search?q=Accra,Ghana&format=json" 

[{"place_id":289735786,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"relation","osm_id":12803764,"lat":"5.5571096","lon":"-0.2012376","class":"boundary","type":"administrative","place_rank":16,"importance":0.5964183313392782,"addresstype":"city","name":"Accra","display_name":"Accra, Korle-Klottey Municipal District, Greater Accra Region, Ghana","boundingbox":["5.5171908","5.6673682","-0.2841318","-0.0548565"]}]


In [32]:
#fix the format

curl -s "https://nominatim.openstreetmap.org/search?q=Kinshasa,Congo&format=json"  | jq

[
  {
    "place_id": 33244447,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
    "osm_type": "node",
    "osm_id": 27043346,
    "lat": "-4.3217055",
    "lon": "15.3125974",
    "class": "place",
    "type": "city",
    "place_rank": 15,
    "importance": 0.6549118197606534,
    "addresstype": "city",
    "name": "Kinshasa",
    "display_name": "Kinshasa, Aketi, Kinshasa, République démocratique du Congo",
    "boundingbox": [
      "-4.4817055",
      "-4.1617055",
      "15.1525974",
      "15.4725974"
    ]
  },
  {
    "place_id": 33281401,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 5646651,
    "lat": "-4.3196982",
    "lon": "15.3424196",
    "class": "boundary",
    "type": "administrative",
    "place_rank": 8,
    "importance": 0.6549118197606534,
    "addresstype": "state",
    "name": "Kinshasa",
    "display_name": "Kinshasa, République démocrat

In [34]:
#API with the Cities.csv file

echo "lat, long, city, addresstype" > Cities_Coordinates.csv
while read city
do    
    result=$(curl -s "https://nominatim.openstreetmap.org/search?q=${city}&format=json" | jq '.[] | [.lat, .lon, .name, .addresstype] | @csv')
    result2=$(echo "${result//\"/}")
    echo "${result2//\\/}"
done < Cities.csv >> Cities_Coordinates.csv